In [2]:
import os
import json
import logging
from datetime import datetime

In [3]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/harsh_arma/.config/sagemaker/config.yaml


In [4]:
boto3.set_stream_logger(name="botocore.credentials", level=logging.WARNING)


In [5]:
def check_aws_configuration():
    try:
        # Attempt to create an AWS S3 client
        s3 = boto3.client('s3')
        
        # List buckets to verify AWS credentials and region
        buckets = s3.list_buckets()
        
        print("AWS Configuration is successful.")
        print("Available S3 Buckets:")
        for bucket in buckets['Buckets']:
            print(f"- {bucket['Name']}")
        
        return True
    except Exception as e:
        print(f"Error: {e}")
        print("AWS Configuration is not successful.")
        return False

# Check AWS configuration
check_aws_configuration()

AWS Configuration is successful.
Available S3 Buckets:
- airbyte-data-bucket
- apple-maps-rank-data
- aws-cloudtrail-logs-173829655778-1bc62e66
- aws-sam-cli-managed-default-samclisourcebucket-ujqrexj3n4l8
- cf-templates-1f8yr71fcychj-us-east-1
- chat-widget-dev-serverlessdeploymentbucket-c2tv25twqqud
- chat-widget-local-serverlessdeploymentbucket-5d5l8p5zoshf
- chrone-app-chat-campaign-jsons
- chrone-biz-keyword-jsons
- chrone-biz-react-website-dev
- chrone-biz-website-react
- chrone-community-data
- chrone-email-campaign-jsons
- chrone-global-assets
- chrone-internal-media-generation-reactjs
- chrone-lambda-layers-zip
- chrone-lead-magnet
- chrone-lead-magnet-dev
- chrone-ml-models
- chrone-public-media
- chrone-sales-rank-extraction-jsons
- chrone-sms-campaign-jsons
- chrone-sp-website
- chrone-sp-website-internal
- chrone-sp-website-media-bank
- chrone-sp-website-pages
- chrone-ui-reactjs
- chrone-website
- chrone-website-edit-application
- chrone.ai
- chrone.app
- chrone.biz
- chr

True

In [6]:
# sess = sagemaker.Session()
sess = sagemaker.Session(boto_session=boto3.Session(region_name='us-east-1'))
region = sess.boto_region_name
print(region)

us-east-1


In [7]:
role_arn = "arn:aws:iam::173829655778:role/sage_make_full_access"

In [8]:
bucket_name = "imagetaggingdemo"
data_path = "img_clf_multilabel_lst"

nclasses = 131
nimgs_train = 25
nepochs = 10
mini_batch_size = 3

train_instance_type = "ml.g4dn.xlarge"
job_name_prefix = "img-clf-multilabel"

In [9]:
train_image_uri = sagemaker.image_uris.retrieve(
    framework="image-classification",
    region=region,
    image_scope="training",
    version="latest",
)
print(train_image_uri)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.


811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:1


In [10]:
s3_output_path = "s3://{}/{}/{}".format(bucket_name, data_path, "model_output")


In [11]:
clf_estimator = sagemaker.estimator.Estimator(
    image_uri=train_image_uri,
    role=role_arn,
    instance_count=1,
    instance_type=train_instance_type,
    volume_size=50,
    max_run=360000,
    input_mode="File",
    output_path=s3_output_path,
    sagemaker_session=sess,
)

In [12]:
clf_estimator.set_hyperparameters(
    num_classes=nclasses,  # update this
    epochs=nepochs,  # update this
    num_training_samples=nimgs_train,  # update this
    mini_batch_size=mini_batch_size,  # update this
    num_layers=18,
    use_pretrained_model=1,
    image_shape="3,224,224",
    resize=256,
    learning_rate=0.001,
    use_weighted_loss=1,
    augmentation_type="crop_color_transform",
    precision_dtype="float32",
    multi_label=1,
)

In [13]:
s3_train_imgs = "s3://{}/{}/{}".format(bucket_name, data_path, "train_imgs")
s3_valid_imgs = "s3://{}/{}/{}".format(bucket_name, data_path, "valid_imgs")
s3_train_annot = "s3://{}/{}/{}".format(bucket_name, data_path, "train_annots")
s3_valid_annot = "s3://{}/{}/{}".format(bucket_name, data_path, "valid_annots")

train_imgs = sagemaker.inputs.TrainingInput(
    s3_train_imgs,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)
valid_imgs = sagemaker.inputs.TrainingInput(
    s3_valid_imgs,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)
train_annot = sagemaker.inputs.TrainingInput(
    s3_train_annot,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)
valid_annot = sagemaker.inputs.TrainingInput(
    s3_valid_annot,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)

data_channels = {
    "train": train_imgs,
    "validation": valid_imgs,
    "train_lst": train_annot,
    "validation_lst": valid_annot,
}

In [14]:
print(sagemaker.__version__)


2.201.0


In [15]:
timestamp = (
    str(datetime.now().replace(microsecond=0)).replace(" ", "-").replace(":", "-")
)
job_name = job_name_prefix + timestamp
print(job_name)

img-clf-multilabel2023-12-20-12-42-28


In [20]:
clf_estimator.fit(inputs=data_channels, logs=True, job_name=job_name)


INFO:sagemaker:Creating training-job with name: img-clf-multilabel2023-12-20-12-42-28


2023-12-20 07:13:53 Starting - Starting the training job...
2023-12-20 07:14:09 Starting - Preparing the instances for training.....
2023-12-20 07:15:45 Downloading - Downloading input data...
2023-12-20 07:16:15 Downloading - Downloading the training image.................
2023-12-20 07:20:51 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
Nvidia gpu devices, drivers and cuda toolkit versions (only available on hosts with GPU):
Wed Dec 20 07:21:04 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memor

In [21]:
print(sagemaker.__version__)
print(sagemaker.__file__)


2.201.0
/home/harsh_arma/Documents/Coding stuff/Intern__/TASK/Image_tagging/ImageClassification/env/lib/python3.10/site-packages/sagemaker/__init__.py


In [22]:
infer_instance_type = "ml.t2.medium"
model_name = job_name
endpoint_name = job_name

In [23]:
clf_predictor = clf_estimator.deploy(
    initial_instance_count=1,
    instance_type=infer_instance_type,
    endpoint_name=endpoint_name,
    model_name=model_name,
)

INFO:sagemaker:Creating model with name: img-clf-multilabel2023-12-20-12-42-28
INFO:sagemaker:Creating endpoint-config with name img-clf-multilabel2023-12-20-12-42-28
INFO:sagemaker:Creating endpoint with name img-clf-multilabel2023-12-20-12-42-28


--------------------!

In [24]:
sgmkr_runt =  boto3.client("sagemaker-runtime", region_name='us-east-1')


In [25]:
with open("test.jpeg", "rb") as image:
        payload = image.read()
        payload = bytearray(payload)
        
# print(payload)        
response = sgmkr_runt.invoke_endpoint(
    EndpointName = endpoint_name,
    ContentType = 'image/jpeg',
    # Accept = "application/json;verbose",
    Body = payload,
)

prediction = json.loads(response['Body'].read().decode())
print(prediction)

[0.0008878662483766675, 5.481954212882556e-05, 0.002953188493847847, 6.155760638648644e-05, 0.18068698048591614, 0.00011905275459866971, 4.637461097445339e-05, 6.802827556384727e-05, 0.0011748867109417915, 0.9865672588348389, 2.0460454834392294e-05, 0.26805317401885986, 0.0003528805682435632, 0.007358838804066181, 0.002235549269244075, 3.920853487215936e-05, 0.0008348595583811402, 0.00547525379806757, 0.9958124756813049, 0.00048548454651609063, 8.116445678751916e-05, 0.001121196779422462, 0.00045517555554397404, 0.0014765721280127764, 0.00025220931274816394, 0.0002324573288206011, 3.046737219847273e-05, 0.00014198978897184134, 0.0008964711450971663, 0.0016279139090329409, 0.996721088886261, 0.00014838941569905728, 0.00011685959907481447, 0.0010491982102394104, 0.000485966942505911, 1.4760559679416474e-05, 6.663609383394942e-05, 0.9927478432655334, 4.381087819638196e-06, 0.00045703156501986086, 3.184110710208188e-06, 0.00010836074943654239, 0.0042686499655246735, 0.004192569758743048, 0

In [29]:
a = [0.0008878662483766675, 5.481954212882556e-05, 0.002953188493847847, 6.155760638648644e-05, 0.18068698048591614, 0.00011905275459866971, 4.637461097445339e-05, 6.802827556384727e-05, 0.0011748867109417915, 0.9865672588348389, 2.0460454834392294e-05, 0.26805317401885986, 0.0003528805682435632, 0.007358838804066181, 0.002235549269244075, 3.920853487215936e-05, 0.0008348595583811402, 0.00547525379806757, 0.9958124756813049, 0.00048548454651609063, 8.116445678751916e-05, 0.001121196779422462, 0.00045517555554397404, 0.0014765721280127764, 0.00025220931274816394, 0.0002324573288206011, 3.046737219847273e-05, 0.00014198978897184134, 0.0008964711450971663, 0.0016279139090329409, 0.996721088886261, 0.00014838941569905728, 0.00011685959907481447, 0.0010491982102394104, 0.000485966942505911, 1.4760559679416474e-05, 6.663609383394942e-05, 0.9927478432655334, 4.381087819638196e-06, 0.00045703156501986086, 3.184110710208188e-06, 0.00010836074943654239, 0.0042686499655246735, 0.004192569758743048, 0.0024690397549420595, 0.00022962753428146243, 0.0002061946870526299, 0.0003174234298057854, 4.3564868974499404e-05, 0.00036894381628371775, 6.664772081421688e-05, 0.0007753618410788476, 0.0002336314064450562, 4.767241625813767e-05, 0.0004663943836931139, 5.234787749941461e-05, 0.00026348113897256553, 0.0007689653430134058, 0.0006981974001973867, 3.0021103157196194e-05, 0.00016293063526973128, 4.860624812863534e-06, 0.0001580565731273964, 4.868546238867566e-05, 0.0006439267308451235, 0.9998080134391785, 0.00012969128147233278, 4.1302377212559804e-05, 0.0010539934737607837, 0.999796450138092, 0.00511931674554944, 0.0016546185361221433, 0.0006390534690581262, 4.097425971849589e-06, 0.00016631543985567987, 5.136618347023614e-05, 0.00010031383135356009, 0.00045943661825731397, 0.000336505618179217, 0.0005575465620495379, 0.003162587294355035, 5.751354910898954e-05, 0.010145796462893486, 0.00016456223966088146, 0.005411835853010416, 0.0002741594216786325, 0.00013906616368331015, 0.0001310165534960106, 0.00015136865840759128, 0.0018777649383991957, 0.0011486487928777933, 0.00039925723103806376, 0.00018965646449942142, 0.00018145961803384125, 0.0001226062886416912, 0.0010564744006842375, 3.603582808864303e-05, 0.003096601227298379, 0.0005554454983212054, 0.00011836599878733978, 0.00012730533489957452, 0.432314932346344, 0.0008061136468313634, 0.00020376262546051294, 0.0007363527547568083, 6.289759767241776e-05, 8.049748430494219e-05, 0.00014818142517469823, 6.760790711268783e-05, 0.0001852557761594653, 0.001419122563675046, 0.000850400363560766, 1.7858452338259667e-05, 0.004336640238761902, 0.0005297379684634507, 0.00022159950458444655, 4.651115523301996e-05, 4.107091808691621e-05, 0.0008665811037644744, 0.003413309110328555, 0.0007104154210537672, 1.7283942725043744e-05, 0.0004377586010377854, 0.00023184421297628433, 0.0025878252927213907, 2.4867855245247483e-05, 3.122911948594265e-05, 0.0010560986120253801, 0.0029581121634691954, 0.008955039083957672, 0.0002827680145855993]
b = ["anesthesia","Upper Body Wax","restorative dentistry","beauty training","Female","Kids","nail services","cosmetic injectables","mens wax","Square","mens colour","Extension services","Old","beauty treatment","portrait photography","grooming","business photography","wispy","Barber","plasma fibrolast","medical services","vitamin injections","wrinkles","microneedling","alternative therapy","wedding services","reflexology","teeth","tatto service","Hair Highlights","Right","body massage","chemical peels","group fitness classes","sugaring","physical therapy","lip treatments","Male","Bohemian Box Braids","piercing services","Locs","body waxing","Landscape","other","dreadlocs","passion twist","Left","cat eye","microblading","none","dermal fillers","sew ins","eyelash services","Gel Nails","lifestyle photography","weight loss services","waxing services","loc services","stretch mark treatment","group services","hair saloons","vajacial","acupunture","Hard Wax","perm","White","energy healing","fitness programs","Braids","Middle","Manicure","cornrows","crystal healing","detoxification services","crochet","eyebrow waxing","Eyebrows","Lower Body Wax","bridal services","Black","Body Contouring","Bridal","Strip Wax","updos","spa","grocery shopping","kids services","bikini wax","hair treatments","wig services","hair styling","artistic services","awakening programs","Dreads","feed ins","makeup services","Eyelash fills","nail polish","facial treatments","beauty services","weaves","Center","Knotless Braids","Brown","cupping therapy","microdermabrasion","personal training","eyebrow services","Pedicure","permanent makeup","skin care services","meditation","iv therepy","Potrait","Eyelash Lift","Acne treatment","yoga and meditation","tanning services","pilates classes","cosmetic dentistry","Box Braids","cooking classes","mens braids","discount services","bohemian twist","barber services","hard","Soft Wax","Facial waxing","light therepy","kinky twist"]
# link a and b
threshold = 0.5  # Set your threshold value here

# Filter elements from list b based on the threshold in list a
filtered_elements = [category for prob, category in zip(a, b) if prob >= threshold]

# Print the filtered elements
print(filtered_elements)

print(len(a))

['Square', 'Barber', 'Right', 'Male', 'White', 'Middle']
131


In [28]:
clf_predictor.delete_endpoint()


INFO:sagemaker:Deleting endpoint configuration with name: img-clf-multilabel2023-12-20-12-42-28


ClientError: An error occurred (ValidationException) when calling the DeleteEndpointConfig operation: Could not find endpoint configuration "img-clf-multilabel2023-12-20-12-42-28".